In [10]:
import ipywidgets as widgets
import os
import io

In [11]:
class Main(widgets.Tab):
    
    
    ## Object oriented approach, inheriting Tab class
    def __init__(self):
        super().__init__()
        
        
        # artifacts from template
        self.button3 = widgets.Button(button_style = 'success', description = 'Self Number 3')
        self.button4 = widgets.Button(button_style = 'success', description = 'Self Number 4')
        
        
        ##########################
        #### FIRST TAB LAYOUT ####
        ##########################
        self.layout1 = widgets.GridspecLayout(4,3)
        
        
        ## Manuscript selection pane
        self.manuscripts = Main.currentManuscripts()
        
        self.selectionKey = {}
        self.selectionNames = []
        for manuscript in self.manuscripts:
            self.selectionNames.append(manuscript[1]['Work'])
            self.selectionKey[self.selectionNames[-1]] = manuscript
        
        self.manuscriptSelection = widgets.Select(
            options=self.selectionNames, # calls
            value=self.selectionNames[0],
            rows=10,
            disabled=False,
            layout=widgets.Layout(height='auto', width='auto')
        )
        
        self.layout1[:3, 1:] = self.manuscriptSelection
        
        ## Info Pane
        infoPane1 = widgets.HTML(value="<h2>Info Pane</h2>\
                                <p>This menu allows you to upload new manuscripts and access previously uploaded manuscripts.\
                                The uploader will accept only .xml and image files.\
                                For testing purposes, it only accepts images right now.<p>",)
        self.layout1[:, 0] = infoPane1
        
        
        
        ## New Manuscript
        self.newMan = Main.create_expandedButton('New Manuscript', 'info')
        self.layout1[3, 1] = self.newMan
        
        self.newMan.on_click(self.on_click_newMan)
        
        
        ## Select Manuscript
        self.selMan = Main.create_expandedButton('Select Manuscript', 'primary')
        self.layout1[3, 2] = self.selMan
        
        self.selMan.on_click(self.on_click_selMan)
        
        
        ###########################
        #### SECOND TAB LAYOUT ####
        ###########################
        self.layout2 = Main.metaSelect(self)
        
        ##########################
        #### THIRD TAB LAYOUT ####
        ##########################
        
        ###########################
        #### FOURTH TAB LAYOUT ####
        ###########################
        
        
        # artifacts from template
        self.layout3 = widgets.Box((self.button3, ))
        self.layout4 = widgets.Box((self.button4, ))
        
        ### Placing layouts in tabbed widget
        self.children = (self.layout1, self.layout2, self.layout3, self.layout4)
        self.set_title(0, 'Manuscript Selection')
        self.set_title(1, 'Manuscript Info')
        self.set_title(2, 'Annotation')
        self.set_title(3, 'Export')

        
        
    ######################
    ### HELPER METHODS ###
    ######################
    
    ### TAB 1 ###
    def on_click_selMan(self, null):
        # Moves the tab to the metadata page with selected manuscript data filled in
        
        # TODO: Make selected manuscript data populate automatically
        
        self.layout2 = Main.metaSelect(self)
        self.children = (self.layout1, self.layout2, self.layout3, self.layout4)
        self.selected_index = 1
    
    def on_click_newMan(self, null):
        self.layout2 = Main.metaNew(self)
        self.children = (self.layout1, self.layout2, self.layout3, self.layout4)
        self.selected_index = 1

    ### TAB 2 ###
    def metaBase(self):
        # This method is used to construct layouts for both new and select manuscript opts.
        # It includes important information for TEI XML and Pinakes work, some for ID
        # Other info included is useful for maintaining a record of the manuscript history
        
        baseLayout = widgets.GridspecLayout(4,2)
        
        ## Authors and Works
        self.author = Main.met_text('Author')
        self.work = Main.met_text('Work')
        baseLayout[0,0] = self.work
        baseLayout[0,1] = self.author
        
        
        ## Cities, Depots, and Odds
        self.country = Main.met_text('Country')
        self.language = Main.met_text('Language')
        baseLayout[1,0] = self.language
        baseLayout[1,1] = self.country
        
        self.city = Main.met_text('City')
        self.institution = Main.met_text('Institution')
        baseLayout[2,0] = self.city
        baseLayout[2,1] = self.institution
        
        ## Century
        centuries = ['1st','2nd','3rd'] + [f'{i}th' for i in range(4,21)]
        self.centuriesWidget = widgets.SelectionRangeSlider(
            options = centuries,
            index = (0,19),
            description = 'Centuries',
            layout = widgets.Layout(height='auto', width='auto')
        )
        baseLayout[3,0] = self.centuriesWidget
        
        ## Move to Annotation
        # TODO: Make the move to annotation button move to the next tab
        self.movToAnnotation = Main.create_expandedButton('Annotation', 'primary')
        baseLayout[3,1] = self.movToAnnotation

        return baseLayout
        
    def met_text(info):
        # This method is called in metaBase
        # To create the text boxes necessary for metadata/info to be added
        return widgets.Text(
            description = info,
            placeholder = info,
            layout = widgets.Layout(
                height='auto',
                width='auto',
                align_items='center')
        )
    
    def metaSelect(self):
        # This method creates a layout when the select manuscript button is used
        selLayout = widgets.GridspecLayout(1,2)
        
        selLayout[0,1] = Main.metaBase(self)
        
        infoPane2 = widgets.HTML(
            value = "<h2>Info Pane</h2>\
                <p>Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas.\
                Vestibulum tortor quam, feugiat vitae, ultricies eget, tempor sit amet, ante.\
                Donec eu libero sit amet quam egestas semper.\
                Aenean ultricies mi vitae est.\
                Mauris placerat eleifend leo.\
                Quisque sit amet est et sapien ullamcorper pharetra.\
                Vestibulum erat wisi, condimentum sed, commodo vitae, ornare sit amet, wisi.\
                Aenean fermentum, elit eget tincidunt condimentum, eros ipsum rutrum orci, sagittis tempus lacus enim ac dui.\
                Donec non enim in turpis pulvinar facilisis.\
                Ut felis.\
                Praesent dapibus, neque id cursus faucibus, tortor neque egestas augue, eu vulputate magna eros eu erat.\
                Aliquam erat volutpat.\
                Nam dui mi, tincidunt quis, accumsan porttitor, facilisis luctus, metus.</p>",
        )
        selLayout[0,0] = infoPane2
        
        return selLayout
        
    
    def metaNew(self):
        # This method creates a layout when the new manuscript button is used
        newLayout = widgets.GridspecLayout(4,2)
        
        # TODO: Make this uploader use only image files
        self.upImages = widgets.FileUpload(
            accept = '',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple = True,  # True to accept multiple files upload else False
            description = 'Upload Manuscript Images',
            layout = widgets.Layout(height='auto', width='auto')
        )
        newLayout[2,0] = self.upImages
        
        # TODO: Make this uploader use only XML files
        self.upTranscripts = widgets.FileUpload(
            accept = '',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
            multiple = True,  # True to accept multiple files upload else False
            description = 'Upload Manuscript Transcriptions',
            layout = widgets.Layout(height='auto', width='auto')
        )
        newLayout[3,0] = self.upTranscripts
        
        infoPane2 = widgets.HTML(
            value = "<h2>Info Pane</h2>\
                <p>Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas.\
                Vestibulum tortor quam, feugiat vitae, ultricies eget, tempor sit amet, ante.\
                Donec eu libero sit amet quam egestas semper.\
                Aenean ultricies mi vitae est.\
                Mauris placerat eleifend leo.\
                Quisque sit amet est et sapien ullamcorper pharetra.\
                Vestibulum erat wisi, condimentum sed, commodo vitae, ornare sit amet, wisi.\
                Aenean fermentum, elit eget tincidunt condimentum, eros ipsum rutrum orci, sagittis tempus lacus enim ac dui.\
                Donec non enim in turpis pulvinar facilisis.\
                Ut felis.\
                Praesent dapibus, neque id cursus faucibus, tortor neque egestas augue, eu vulputate magna eros eu erat.\
                Aliquam erat volutpat.\
                Nam dui mi, tincidunt quis, accumsan porttitor, facilisis luctus, metus.</p>",
        )
        newLayout[:2,0] = infoPane2
        
        newLayout[:,1] = Main.metaBase(self)
        
        return newLayout
    
        
    ### ALL ###
    def create_expandedButton(description, button_style):
        return widgets.Button(
            description = description,
            button_style = button_style,
            layout = widgets.Layout(height='auto', width='auto')
        )
    
    def metadataDictionary(self):
        metadata = {
            'Work': self.work.value,
            'Author': self.author.value,
            'Language': self.author.value,
            'Country': self.country.value,
            'City': self.city.value,
            'Institution': self.institution.value,
            'Centuries': self.centuriesWidget.value
        }
        return metadata
    
    def currentManuscripts():
        # If this is run on any computer, it will have a unique structure
        current = os.getcwd()
        manuscriptDirectory = os.path.join(current,'manuscripts')

        # This is necessary to keep directories accessible. Without os.path.join, we can't keep a full directory name and access files inside specific directories
        directories = []
        for path in os.listdir(manuscriptDirectory):
            directories.append(os.path.join(manuscriptDirectory,path))

        # This is necessary to store metadata from .cfg files
        manuscriptMetadata = []

        # This is necessary to search each directory in the manuscripts folder
        for directory in directories:
            # This looks through each file in a given directory
            for file in os.listdir(directory):
                # This opens config files and reads metadata from them
                if file.endswith('.cfg'):
                    fileDirectory = os.path.join(directory,file)
                    f = open(fileDirectory, 'r')
                    metadata = {}

                    for line in f:
                        key, value = line.split(':')
                        metadata[key] = value[:-1]

                    manuscriptMetadata.append([directory, metadata])

        os.chdir(current)
        return manuscriptMetadata

In [13]:
tab2 = Main()

tab2

Main(children=(GridspecLayout(children=(Select(layout=Layout(grid_area='widget001', height='auto', width='auto…

In [8]:
def currentManuscripts():
    # If this is run on any computer, it will have a unique structure
    current = os.getcwd()
    manuscriptDirectory = os.path.join(current,'manuscripts')
    
    # This is necessary to keep directories accessible. Without os.path.join, we can't keep a full directory name and access files inside specific directories
    directories = []
    for path in os.listdir(manuscriptDirectory):
        directories.append(os.path.join(manuscriptDirectory,path))
    
    # This is necessary to store metadata from .cfg files
    manuscriptMetadata = []
    
    # This is necessary to search each directory in the manuscripts folder
    for directory in directories:
        # This looks through each file in a given directory
        for file in os.listdir(directory):
            # This opens config files and reads metadata from them
            if file.endswith('.cfg'):
                fileDirectory = os.path.join(directory,file)
                f = open(fileDirectory, 'r')
                metadata = {}

                for line in f:
                    key, value = line.split(':')
                    metadata[key] = value[:-1]

                manuscriptMetadata.append([directory, metadata])
    
    os.chdir(current)
    return manuscriptMetadata